In [ ]:
import time
import pandas as pd
from datetime import datetime
import joblib
import os.path

#Firebase Libraries
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

#Machine Leearning Libraries
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets

SERIAL_NUM = 123

#Fetch the service account key JSON file contents
cred = credentials.Certificate("serviceAccountKey.json")

#initialize the app if not already
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://smartyblinds-default-rtdb.firebaseio.com/'
    })

#Automated process, only if we have trained a model

while(1):
    
    if(os.path.isfile('Tree_Model_'+ str(SERIAL_NUM) +'.sav')):

        #Retrieve Model
        dtree = joblib.load('Tree_Model_'+ str(SERIAL_NUM) +'.sav')

        #Get Current sensor data
        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/Light')
        current_light = ref_event.get()

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/Temp')
        current_temperature = ref_event.get()

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/RTC')
        current_RTC = ref_event.get()
        dateOBJ = datetime.strptime(str(current_RTC), "%Y-%m-%d %H:%M:%S")

        day = dateOBJ.weekday()

        month = dateOBJ.month

        minutes = (dateOBJ.hour * 60) + (dateOBJ.minute)
        
        print("Light, Temp, Day, Min")
        
        current_light, current_temperature, day, minutes = input("Current test: ").split()
        
        current_test = [[current_light,current_temperature,day,minutes]]

        #print(current_test)

        #Current sensor data is tested with the model
        new_output = dtree.predict(current_test)

        print(new_output)

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Blind_State')
        blind_event = ref_event.get()

        if (new_output.item(0) != blind_event):

            #control blinds automatically
            ref = db.reference('Blinds/' + str(SERIAL_NUM) + '/')
            ref.update({"Operation": "Auto"})
            ref.update({"Blind_State": new_output.item(0)})

Light, Temp, Day, Min
